# Bottom-Up Flavor 

Here, multiplicity is inferred from the given parameters/hyperparameters

In [ ]:
%pip install -e ../cybercomp-python-sdk

In [1]:
from pathlib import Path

from cybercomp import Completions

# generate statically typed objects to define experiments
completions = Completions(
    server_url="http://127.0.0.1:8765",
    base_path=Path("./completions"),
)
completions.sync()

Connected to cybercomp server on http://127.0.0.1:8765
[Type] Loaded dl_dataset
[Type] Loaded dl_dataset_name
[Type] Loaded dl_epochs
[Type] Loaded dl_eval_metric
[Type] Loaded dl_model
[Type] Loaded dl_model_name
[Type] Loaded dl_model_weights
[Type] Loaded dl_optimizer
[Type] Loaded dl_probe_layer
[Type] Loaded dl_scheduler
[Type] Loaded neuro_3D_distance
[Type] Loaded neuro_3D_subnet
[Type] Loaded neuro_connection_info
[Type] Loaded neuro_current_params
[Type] Loaded neuro_current_stim
[Type] Loaded neuro_cx_cx_g_ampa0
[Type] Loaded neuro_dat
[Type] Loaded neuro_field_file_0
[Type] Loaded neuro_field_file_1
[Type] Loaded neuro_field_file_2
[Type] Loaded neuro_field_file_3
[Type] Loaded neuro_field_file_4
[Type] Loaded neuro_field_file_5
[Type] Loaded neuro_field_file_6
[Type] Loaded neuro_field_file_7
[Type] Loaded neuro_graf_cx
[Type] Loaded neuro_graf_cx6
[Type] Loaded neuro_graf_cxa
[Type] Loaded neuro_graf_in
[Type] Loaded neuro_graf_in6
[Type] Loaded neuro_graf_ina
[Type] Loade

In [2]:
from cybercomp.functional import parameter, hyperparameter, observation, step, experiment
from cybercomp import Runtime
from completions import models, engines
from completions.types import *

In [4]:
# define a simple experiment
ex1 = experiment(
    step(models.make_network(), engines.bazhenov_labs_engine(), name="make_network"),
    step(models.stim_current(), engines.bazhenov_labs_engine(), name="simulate_network"),
)
print(ex1)

name=[make_network+simulate_network], steps=[make_network>>simulate_network]


In [5]:
# alternative definition
s1 = step(models.make_network(), engines.bazhenov_labs_engine(), name="make_network")
s2 = step(models.stim_current(), engines.bazhenov_labs_engine(), name="simulate_network")
ex1 = s1 >> s2
print(ex1)

name=[make_network+simulate_network], steps=[make_network>>simulate_network]


In [6]:
# define a runtime (local)
r = Runtime()

In [6]:
# run the simple experiment
args = (
    parameter(neuro_network_config, "samples/sleep_stage_transition/network.cfg"),
    parameter(neuro_connection_info, "samples/sleep_stage_transition/connection_info"),
    hyperparameter(neuro_current_params, "samples/sleep_stage_transition/params.txt"),
)
ex1.run(args, runtime=r)
ex1.fetch(
    args,
    runtime=r,
    observables=[
        observation(neuro_time_in, "samples/sleep_stage_transition/neuro_time_in"),
    ],
)

In [ ]:
# create a grid search type experiment
s1 = step(models.make_network(), engines.bazhenov_labs_engine(), name="make_network")
s2 = step(models.stim_current(), engines.bazhenov_labs_engine(), name="simulate_network")
ex1 = s1 >> s2
replicate(ex1, )

In [ ]:
# define a runtime for the experiments
runtime = Runtime()

# experiments can be chained into a larger experiment
# here, the parameters and observables of the second experiment are resolved
# from the second experiment, and can be overridden.
# the operator validates the parameters and warns if any parameter is missing.
ex1 > ex2 > (ex3 | ex4 | ex5) > ex6

ex6 = () # ex1 > ex2 > (ex3 | ex4 | ex5) > ex6 must be inferred

seq1 = experiment(
    name="bazhenov_network",
    context=(ex1 >> ex2,)
)
print(isinstance(seq1, Experiment))

In [ ]:
%run_remote

output = ex1 > ex2 > ex3

In [ ]:
%run_local

output > ex4

In [7]:
range_P = [
    P.Range(models.stim_current.neuro_connection_info, 0.1, 1.0, 10),
    P.Range(models.stim_current.neuro_current_params, 0.1, 1.0, 10),
]
range_H = [
    H.Range(engines.bazhenov_labs_engine.neuro_current_params, 0.1, 1.0, 10),
]

In [ ]:
sweep1 = Experiment.Sweep("bazhenov_network_grid_search", (seq1,), range_P, range_H, []) # change syntax
sweep1.run(runtime)
exChain_obs = sweep1.fetch(runtime)

In [ ]:
# run sweep
for p in range_P:
    for h in range_H:
        

In [ ]:
ex12 = Collection(
    name="simulate_bhazenov_network",
    models=[models.make_network, models.stim_current],
    engine=engines.bazhenov_labs_engine,
    parameters=[
        models.make_network.neuro_network_config("samples/sleep_stage_transition/params.txt"),
        models.stim_current.neuro_current_params("samples/sleep_stage_transition/connection_info")
    ],
    parameters={
        "neuro_network_config": P("samples/sleep_stage_transition/network.cfg"),
        "neuro_current_params": P("samples/sleep_stage_transition/params.txt"),
    },
    hyperparameters={
        "neuro_current_params": H("samples/sleep_stage_transition/params.txt"),
    },
    observables={
        "neuro_time_cx6": O("observables/time_cx6")
    },
)

runtime = Runtime()
ex2.execute(runtime)
ex2_obs = ex2.gather_observables(runtime)

## POC - double pendulum simulation
## Real - replicate simone's experiments


# the intersecting observables are piped into the params of the next experiment
# only the intersecting observables are shown in autocompletion

experiment_2 = create_new_experiment_from(experiment_1)
experiment_2.name = "<>"
experiment_2.parameters["neuro/network_config"] = "new-value"
experiment.check_existing()

experiment.observables

experiemnt.validate()
experiment.run(
hpc_recipe={}
)

# !!! [collection] - a set of experiments with common observables

# observables may be a huge list, so need not provide everytime when its

implictly discoverable

# to get experiments run with different observables

collection = create_collection(
model="name_of_model",
parameters={
"neuro/network_config": [],
},
)

# the collection experiments are pulled from the db

collection = create_collection(
model=["model1", "model2", ...],
parameters={
"neuro/network_config": [],
},
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

# all experiments sharing the same observables

collection = create_collection(
observables={
"neuro/network_config": [],
},
) -> [list of experiments]

collection = [experiment_1, experiment_2]

# experiment collection

# example of experiment chaining (top-to-bottom mro)

# example 1

experiment = Experiment(
experiment_1,
experiment_2,
)

# example 2

experiment = Experiment(
[experiment_2, experiment_3, .....], #
experiment_1, #
)

#

[
exp2 -> exp1,
exp3 -> exp1,
]

# example 3

experiment = Experiment(
[experiment_2, ...collection.experiments],
experiment_1,
)

In [ ]:
# analysis part =========================

# takes a collection as input,

# and runs some function over the observables on that

# collection

# a primitive form of experiment using a collection of experiments as input

analysis = Analysis(
collection=[],
function={

    }

)

analysis = experiment